# Intro To Handling Dates In Pandas
Properly handling dates in Pandas can be very useful.  For example, let's say you read in a csv of temperature data with dates and you'd like to find the monthly mean temperature.  If the date column is already recognized, by pandas, as a date type, then extracting the month from the column will be very easy.  If the date column is, instead, read in as a string type, then it will be more difficult to extract the month.

## Notebook Outline
* <a href='#parsedates'>Using the parse dates .read_csv() to automatically read in dateformats</a>
* <a href='#todatetime'>Using .to_datetime() to convert a column to a pandas datetime format</a>
* <a href='#dtattribute'>Using .dt on datetime columns</a>
* <a href='#minmaxsum'>Introduction the .min(), .max() and .sum() methods</a>

In [1]:
import pandas as pd
import os

<a name=parsedates></a>
# Using the parse_dates argument in .read_csv() to automatically read in date formats
When reading in a file, we can specify which columns, or which combinations of columns, we would like read in as a datetime type. For this example, I am going to introduce a new data file 'LaborSheetData.csv'. This file contains real data from a very popular fast food store. Every hour, the shift manager must enter some key data in this file, like drive through times and sales for the past hour. This will be a good dataset for us to explore in some of our lectures.

First we will load the data. Remember that you will need to change the path to point to where you place the file on your computer, after you download it.

In [2]:
filepath = os.path.join(os.getcwd(), 'data', 'LaborSheetData.csv')
laborSheetData = pd.read_csv(filepath)

#### Use the .head() method to get a look at the data, and the .info method to see the data types

In [3]:
laborSheetData.head(2)

,Store,Manager,Date,Hour,Projected Sales,Sales,DT TTL,KVS Total,Hrs +/- (Actual vs Need),Accum Hrs +/- (Act vs Need),Reason for +/- Labor,Reason for High TTL's,Manager Entering Data,TimeStamp
0,4007,Monica H,2/4/17,7:00:00,281.0,296.0,192,117,-1.00,1.00,NaN,NaN,NaN,2/4/17 7:10
1,4007,David H,2/4/17,8:00:00,670.0,347.0,111,97,2.39,2.39,NaN,NaN,NaN,2/4/17 8:04


In [4]:
laborSheetData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13178 entries, 0 to 13177
Data columns (total 14 columns):
Store                          13178 non-null int64
Manager                        13175 non-null object
Date                           13178 non-null object
Hour                           13178 non-null object
Projected Sales                13178 non-null float64
Sales                          13178 non-null float64
DT TTL                         13178 non-null int64
KVS Total                      13178 non-null int64
Hrs +/- (Actual vs Need)       13178 non-null float64
Accum Hrs +/- (Act vs Need)    13178 non-null float64
Reason for +/- Labor           396 non-null object
Reason for High TTL's          260 non-null object
Manager Entering Data          0 non-null float64
TimeStamp                      13178 non-null object
dtypes: float64(5), int64(3), object(6)
memory usage: 1.4+ MB


#### Note that data types of the Date and TimeStamp columns
The data types are object, which means they were read in as strings. Let's double check that by grabbing a value from the data column using the .loc() method and finding its type with the type function.

In [5]:
type(laborSheetData.loc[0, 'Date'])

str

#### Now, we will use the parse_dates argument to automatically read the date columns in as date types.
All we need to do is pass a list of the columns we would like pandas to try and automatically decipher as date or time objects to the parse_dates argument.  If you look above at the output from .info, you will see that the date or datetime columns are columns 2 and 13 (remember that counting starts at 0).

Note that this will cause the read_csv() method to take a little bit longer to complete.

In [6]:
# add the parse_date attribute
laborSheetData = pd.read_csv(filepath, parse_dates=[2, 13])

# or use the column names
laborSheetData = pd.read_csv(filepath, parse_dates=['Date', 'TimeStamp'])

In [7]:
laborSheetData.head(2)

,Store,Manager,Date,Hour,Projected Sales,Sales,DT TTL,KVS Total,Hrs +/- (Actual vs Need),Accum Hrs +/- (Act vs Need),Reason for +/- Labor,Reason for High TTL's,Manager Entering Data,TimeStamp
0,4007,Monica H,2017-02-04,7:00:00,281.0,296.0,192,117,-1.00,1.00,NaN,NaN,NaN,2017-02-04 07:10:00
1,4007,David H,2017-02-04,8:00:00,670.0,347.0,111,97,2.39,2.39,NaN,NaN,NaN,2017-02-04 08:04:00


#### Now use .info() to check our datatypes.
Note that the 'Date' and 'TimeStamp' columns are now datetime types!

In [8]:
laborSheetData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13178 entries, 0 to 13177
Data columns (total 14 columns):
Store                          13178 non-null int64
Manager                        13175 non-null object
Date                           13178 non-null datetime64[ns]
Hour                           13178 non-null object
Projected Sales                13178 non-null float64
Sales                          13178 non-null float64
DT TTL                         13178 non-null int64
KVS Total                      13178 non-null int64
Hrs +/- (Actual vs Need)       13178 non-null float64
Accum Hrs +/- (Act vs Need)    13178 non-null float64
Reason for +/- Labor           396 non-null object
Reason for High TTL's          260 non-null object
Manager Entering Data          0 non-null float64
TimeStamp                      13178 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(5), int64(3), object(4)
memory usage: 1.4+ MB


#### Now let's use the parse_dates argument to _combine_ two (or more) columns into one datetime. 
For this, I need to give you some information about this data. The 'TimeStamp' column let's us know when the data was entered, but the 'Date' + 'Hour' column let us know what hour the data is for.  So it would be great if we could combine the Date and Hour columns into one datetime column!

We can do this using the parse_dates argument. All we need to do is include a list, of the columns we want to combine, as one of the items in the list that we we pass to the parse_dates argument.  

Note that pandas will create a _new_ column called 'Date_Hour' that combines the 'Date' and 'Hour' columns two columns.

In [9]:
# add parse_dates
laborSheetData = pd.read_csv(filepath, parse_dates=[[2, 3], 13])

In [10]:
laborSheetData.head(2)

,Date_Hour,Store,Manager,Projected Sales,Sales,DT TTL,KVS Total,Hrs +/- (Actual vs Need),Accum Hrs +/- (Act vs Need),Reason for +/- Labor,Reason for High TTL's,Manager Entering Data,TimeStamp
0,2017-02-04 07:00:00,4007,Monica H,281.0,296.0,192,117,-1.00,1.00,NaN,NaN,NaN,2017-02-04 07:10:00
1,2017-02-04 08:00:00,4007,David H,670.0,347.0,111,97,2.39,2.39,NaN,NaN,NaN,2017-02-04 08:04:00


In [13]:
laborSheetData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13178 entries, 0 to 13177
Data columns (total 13 columns):
Date_Hour                      13178 non-null datetime64[ns]
Store                          13178 non-null int64
Manager                        13175 non-null object
Projected Sales                13178 non-null float64
Sales                          13178 non-null float64
DT TTL                         13178 non-null int64
KVS Total                      13178 non-null int64
Hrs +/- (Actual vs Need)       13178 non-null float64
Accum Hrs +/- (Act vs Need)    13178 non-null float64
Reason for +/- Labor           396 non-null object
Reason for High TTL's          260 non-null object
Manager Entering Data          0 non-null float64
TimeStamp                      13178 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(5), int64(3), object(3)
memory usage: 1.3+ MB


## Let's see another example using our weather data.
Firs we will load the weather data and use .head() to look at the data.  Note that the first four columns are Year, Month, Day, and Hour.

In [11]:
filepath = os.path.join(os.getcwd(), 'data', 'Philadelphia_Pennsylvania_USA/724080-13739-2001')

headers = ['Year', 'Month', 'Day', 'Hour', 'Air Temp', 'Dew Point Temp', 'Sea Level Pressure',
           'Wind Direction', 'Wind Speed Rate',
           'Sky Condition Total Coverage Code',
           'Liquid Precipitation Depth Dimension - 1Hr Duration',
           'Liquid Precipitation Depth Dimension - Six Hour Duration']
weatherData = pd.read_csv(filepath, delim_whitespace=True,
                          names=headers)

In [15]:
weatherData.head(1)

,Year,Month,Day,Hour,Air Temp,Dew Point Temp,Sea Level Pressure,Wind Direction,Wind Speed Rate,Sky Condition Total Coverage Code,Liquid Precipitation Depth Dimension - 1Hr Duration,Liquid Precipitation Depth Dimension - Six Hour Duration
0,2001,1,1,0,-6,-94,10146,280,57,2,0,-9999


### In Class Exercise:
#### Use parse_dates to reload the data and combine the first four columns into one datetime column

In [12]:
weatherData = pd.read_csv(filepath, delim_whitespace=True,
                          names=headers, parse_dates=[[0, 1, 2, 3]])

In [17]:
weatherData.head(2)

,Year_Month_Day_Hour,Air Temp,Dew Point Temp,Sea Level Pressure,Wind Direction,Wind Speed Rate,Sky Condition Total Coverage Code,Liquid Precipitation Depth Dimension - 1Hr Duration,Liquid Precipitation Depth Dimension - Six Hour Duration
0,2001-01-01 00:00:00,-6,-94,10146,280,57,2,0,-9999
1,2001-01-01 01:00:00,-11,-94,10153,280,57,4,0,-9999


In [18]:
weatherData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8758 entries, 0 to 8757
Data columns (total 9 columns):
Year_Month_Day_Hour                                         8758 non-null datetime64[ns]
Air Temp                                                    8758 non-null int64
Dew Point Temp                                              8758 non-null int64
Sea Level Pressure                                          8758 non-null int64
Wind Direction                                              8758 non-null int64
Wind Speed Rate                                             8758 non-null int64
Sky Condition Total Coverage Code                           8758 non-null int64
Liquid Precipitation Depth Dimension - 1Hr Duration         8758 non-null int64
Liquid Precipitation Depth Dimension - Six Hour Duration    8758 non-null int64
dtypes: datetime64[ns](1), int64(8)
memory usage: 615.9 KB


<a name=todatetime></a>
# Using pandas.to_datetime() to convert a column of strings to a columns of DateTime objects
Sometimes, we need to convert a column after we read in the data. Maybe we have created the column during data processing. We can do this with the to_datetime() function in the pandas package.

I am going to read in the labor sheet data again, without using parse_dates.

In [13]:
filepath = os.path.join(os.getcwd(), 'data', 'LaborSheetData.csv')
laborSheetData = pd.read_csv(filepath)
laborSheetData.head()

,Store,Manager,Date,Hour,Projected Sales,Sales,DT TTL,KVS Total,Hrs +/- (Actual vs Need),Accum Hrs +/- (Act vs Need),Reason for +/- Labor,Reason for High TTL's,Manager Entering Data,TimeStamp
0,4007,Monica H,2/4/17,7:00:00,281.0,296.0,192,117,-1.00,1.00,NaN,NaN,NaN,2/4/17 7:10
1,4007,David H,2/4/17,8:00:00,670.0,347.0,111,97,2.39,2.39,NaN,NaN,NaN,2/4/17 8:04
2,4007,David H,2/3/17,9:00:00,784.0,649.0,143,73,0.18,0.18,NaN,NaN,NaN,2/3/17 18:41
3,4007,David H,2/4/17,10:00:00,852.0,750.0,158,85,0.75,3.25,NaN,NaN,NaN,2/4/17 10:06
4,4007,Zoey D,2/5/17,5:00:00,54.0,108.0,104,45,-1.50,-1.50,NaN,NaN,NaN,2/5/17 10:57


In [ ]:
laborSheetData.info()

#### Use to_datetime to convert the 'TimeStamp' column to a column of datetime objects.
Note that to_datetime can not combine multiple columns (except under a few specific circumstances) so we can not use it to combine the 'Date' and 'Hour' columns.

In [14]:
laborSheetData.loc[:, 'TimeStamp'] = pd.to_datetime(laborSheetData['TimeStamp'])

In [15]:
laborSheetData['Date_Hour'] = laborSheetData['Date'] + ' ' + laborSheetData['Hour']
laborSheetData.head(1)

,Store,Manager,Date,Hour,Projected Sales,Sales,DT TTL,KVS Total,Hrs +/- (Actual vs Need),Accum Hrs +/- (Act vs Need),Reason for +/- Labor,Reason for High TTL's,Manager Entering Data,TimeStamp,Date_Hour
0,4007,Monica H,2/4/17,7:00:00,281.0,296.0,192,117,-1.0,1.0,NaN,NaN,NaN,2017-02-04 07:10:00,2/4/17 7:00:00


In [16]:
laborSheetData.loc[:, 'Date_Hour'] = pd.to_datetime(laborSheetData['Date'] + ' ' + laborSheetData['Hour'])

In [17]:
laborSheetData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13178 entries, 0 to 13177
Data columns (total 15 columns):
Store                          13178 non-null int64
Manager                        13175 non-null object
Date                           13178 non-null object
Hour                           13178 non-null object
Projected Sales                13178 non-null float64
Sales                          13178 non-null float64
DT TTL                         13178 non-null int64
KVS Total                      13178 non-null int64
Hrs +/- (Actual vs Need)       13178 non-null float64
Accum Hrs +/- (Act vs Need)    13178 non-null float64
Reason for +/- Labor           396 non-null object
Reason for High TTL's          260 non-null object
Manager Entering Data          0 non-null float64
TimeStamp                      13178 non-null datetime64[ns]
Date_Hour                      13178 non-null datetime64[ns]
dtypes: datetime64[ns](2), float64(5), int64(3), object(5)
memory usage: 1.5+ MB


<a name=dtattribute></a>
# Using .dt to access and operate on the datetime objects in a column
I am sure you are wondering why we went to the trouble to convert the columns to the datetime data type. Well it let's us manipulate the datetimes much more easily. Let's see some basics below:

First I read the data back in, correctly use the parse_dates argument.

In [18]:
laborSheetData = pd.read_csv(filepath, parse_dates=[[2, 3], 13])
laborSheetData.head(2)

,Date_Hour,Store,Manager,Projected Sales,Sales,DT TTL,KVS Total,Hrs +/- (Actual vs Need),Accum Hrs +/- (Act vs Need),Reason for +/- Labor,Reason for High TTL's,Manager Entering Data,TimeStamp
0,2017-02-04 07:00:00,4007,Monica H,281.0,296.0,192,117,-1.00,1.00,NaN,NaN,NaN,2017-02-04 07:10:00
1,2017-02-04 08:00:00,4007,David H,670.0,347.0,111,97,2.39,2.39,NaN,NaN,NaN,2017-02-04 08:04:00


#### Use .dt to access the datetime properties, and then use .month to get the month of each value in the column

In [19]:
# add .dt.month.head(2)
laborSheetData['Date_Hour'].dt.month.head(2)

0    2
1    2
Name: Date_Hour, dtype: int64

#### Use .dt and .year to get the year of each columns

In [20]:
# add .dt.year.head(2)
laborSheetData['Date_Hour'].dt.year.head(2)

0    2017
1    2017
Name: Date_Hour, dtype: int64

#### Use .dt and .day to get the day of each columns

In [21]:
# add .dt.day.head(2)
laborSheetData['Date_Hour'].dt.day.head(2)

0    4
1    4
Name: Date_Hour, dtype: int64

#### Use .dt and .hour to get the hour of each columns

In [23]:
# add .dt.hour.head(2)
laborSheetData['Date_Hour'].dt.hour.head(2)

0    7
1    8
Name: Date_Hour, dtype: int64

#### Use .dt and .hour and .value_counts() to get the row counts for each hour; do some hours get recorded less than others?

In [24]:
# add .dt.hour.value_counts()
laborSheetData['Date_Hour'].dt.hour.value_counts()

8     806
7     806
9     794
10    784
12    774
13    764
11    762
6     749
14    745
17    744
16    741
18    741
15    738
19    694
20    664
21    627
22    552
23    465
0     210
5       9
1       7
2       1
3       1
Name: Date_Hour, dtype: int64

#### Introducing .sort_index().  The .sort_index() method will sort the index of a dataframe (while reordering the rows appropriately)

Note that the index of the value_counts() results is made up of the values that you are counting.

In [25]:
laborSheetData['Date_Hour'].dt.hour.value_counts().sort_index()

0     210
1       7
2       1
3       1
5       9
6     749
7     806
8     806
9     794
10    784
11    762
12    774
13    764
14    745
15    738
16    741
17    744
18    741
19    694
20    664
21    627
22    552
23    465
Name: Date_Hour, dtype: int64

<a name=minmaxsum></a>
## Introducing the .min(), .max() and .sum() methods.
You can use these methods on any column, or dataframe to get the min, max, and sum of all numerical columns. You can also use the .min() and .max() methods to get the min and max of datetime columns

#### Use .min() and .max() to find the earliest and latest year in the data.

In [26]:
print(laborSheetData['Date_Hour'].min())
print(laborSheetData['Date_Hour'].max())

2017-01-22 22:00:00
2017-09-25 18:00:00


#### Combine the use of .loc, .dt, and .sum() to find the total sales In August across all stores.

In [ ]:
laborSheetData.loc[laborSheetData['Date_Hour'].dt.month == 8, 'Sales'].sum()

#### Selecting rows greater than (or less than) a date.
Getting rows greater than, or less than (or some combination of logic tests) is fairly easy.  You use your normal logic tests: >, <, ==, etc...  and pass the value you'd like to test against as a string.  See the example below, note how we are able to use the string '2017-08-01' to get all rows with a value in the 'TimeStamp' column greater than '2017-08-01'. Easy!

In [ ]:
laborSheetData.loc[laborSheetData['TimeStamp'] > '2017-08-01', :].head()

## In Class Exercise:
Create a cell below and explore the dataset by selecting rows by the TimeStamp or Date_Hour columns

# Changing the time zone of a datetime column
Pandas includes some easy functionality to convert the timezone of a datetime column.  The first thing we need to do is 'localize' the column - this means defining what timezone the original data is in.  They data we have been using happens to be from fast food location on the west coast, so we will localize the timestamp column to the 'US/Pacific' timezone by using the `tz_localize()` method.

You can find a list of all accepted timezones in the top answer to this stackovrflow question: 
https://stackoverflow.com/questions/13866926/python-pytz-list-of-timezones

In [ ]:
laborSheetData['TimeStamp'].dt.tz_localize('US/Pacific').head()

In [ ]:
laborSheetData.loc[:, 'TimeStamp'] = laborSheetData['TimeStamp'].dt.tz_localize('US/Pacific')

In [ ]:
laborSheetData.head()

In [ ]:
laborSheetData.loc[:, 'TimeStamp_UTC'] = laborSheetData['TimeStamp'].dt.tz_convert('UTC')

In [ ]:
laborSheetData.head()

# Using .date_range() to create a DateTime index

We can easily create a DateTime index by using the .date_range() method. We just need to pass a start date, and end date (or a number of periods) and a frequency (the amount of time between each value in the series of values).  Possible values for freq are:
* s (for seconds)
* min (for minutes)
* H (for hours)
* D (for days)
* A (for annual, ending on the end of a year)
* AS (for annual, ending on the start of a year)
* You can also do multiples, i.e. 3H for a step of 3 hours.
* You can also use something called a DateOffset object for more custom steps, but that is beyond the scope of this course.

Note that, once you create an index, you can use it as a row index or column in a dataframe.

The docs for this method are here: <https://pandas.pydata.org/pandas-docs/stable/generated/pandas.date_range.html>

Let's look at some examples:

#### Create a DatetimeIndex with a start date of 2001-01-01, an end date of 2002-01-01 and a step of 3 hours.

In [27]:
pd.date_range(start='2001-01-01', end='2002-01-01', freq='3H')

DatetimeIndex(['2001-01-01 00:00:00', '2001-01-01 03:00:00',
               '2001-01-01 06:00:00', '2001-01-01 09:00:00',
               '2001-01-01 12:00:00', '2001-01-01 15:00:00',
               '2001-01-01 18:00:00', '2001-01-01 21:00:00',
               '2001-01-02 00:00:00', '2001-01-02 03:00:00',
               ...
               '2001-12-30 21:00:00', '2001-12-31 00:00:00',
               '2001-12-31 03:00:00', '2001-12-31 06:00:00',
               '2001-12-31 09:00:00', '2001-12-31 12:00:00',
               '2001-12-31 15:00:00', '2001-12-31 18:00:00',
               '2001-12-31 21:00:00', '2002-01-01 00:00:00'],
              dtype='datetime64[ns]', length=2921, freq='3H')

#### Create a DatetimeIndex with a start date of 2001-01-01 and a  frequency of 1 day that continues for 100 periods.

In [28]:
pd.date_range(start='2001-01-01', periods=100, freq='D')

DatetimeIndex(['2001-01-01', '2001-01-02', '2001-01-03', '2001-01-04',
               '2001-01-05', '2001-01-06', '2001-01-07', '2001-01-08',
               '2001-01-09', '2001-01-10', '2001-01-11', '2001-01-12',
               '2001-01-13', '2001-01-14', '2001-01-15', '2001-01-16',
               '2001-01-17', '2001-01-18', '2001-01-19', '2001-01-20',
               '2001-01-21', '2001-01-22', '2001-01-23', '2001-01-24',
               '2001-01-25', '2001-01-26', '2001-01-27', '2001-01-28',
               '2001-01-29', '2001-01-30', '2001-01-31', '2001-02-01',
               '2001-02-02', '2001-02-03', '2001-02-04', '2001-02-05',
               '2001-02-06', '2001-02-07', '2001-02-08', '2001-02-09',
               '2001-02-10', '2001-02-11', '2001-02-12', '2001-02-13',
               '2001-02-14', '2001-02-15', '2001-02-16', '2001-02-17',
               '2001-02-18', '2001-02-19', '2001-02-20', '2001-02-21',
               '2001-02-22', '2001-02-23', '2001-02-24', '2001-02-25',
      

#### Create a DatetimeIndex with a start date of 1970-01-01, that continues for 10 periods, and has an annual frequency - on the first day of each year.

In [29]:
pd.date_range(start='1970-01-01', periods=10, freq='AS')

DatetimeIndex(['1970-01-01', '1971-01-01', '1972-01-01', '1973-01-01',
               '1974-01-01', '1975-01-01', '1976-01-01', '1977-01-01',
               '1978-01-01', '1979-01-01'],
              dtype='datetime64[ns]', freq='AS-JAN')

#### Create a DatetimeIndex with a start date of 1970-01-01, that continues for 10 periods, and has an annual frequency - on the last day of each year.

In [30]:
pd.date_range(start='1969-12-31', periods=10, freq='A')

DatetimeIndex(['1969-12-31', '1970-12-31', '1971-12-31', '1972-12-31',
               '1973-12-31', '1974-12-31', '1975-12-31', '1976-12-31',
               '1977-12-31', '1978-12-31'],
              dtype='datetime64[ns]', freq='A-DEC')

#### Create a DatetimeIndex with a start date of 2001-01-01 and a step size of 1 day that continues for 100 steps.

In [ ]:
pd.date_range(start='1969-12-31', periods=10, freq='A-AUG')

### In Class Exercise:
Please add a cell below and create three DatetimeInidices of your choosing.

## Question or Comments About This Notebook?
Feel free to contact me via my LinkedIn: https://www.linkedin.com/in/william-j-henry <br>
You can also email me at will@henryanalytics.com <br>